Electronic Health Records - MIMIC-III
=========================



#### Loading libraries



Those are the libraries that might be useful. 



In [ ]:
library(dplyr)
library(tidyr)
library(tibble)
library(lubridate)
library(readr)
library(stringr)
library(data.table)
library(odbc)
library(RMariaDB)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last




#### Connecting to the database



As you know, to connect to the database, we must specify an object with the specific details of the connection. In this exercise, you will use the following object, `con`, to connect. **Place your username and password on the designed locations**.



In [2]:
# Get the credentials file
# It looks like:
################
#username=your.user.name
#password=your.password.assigned
################
creds_file = "creds.txt"

In [3]:
# Set the username and password
creds <- readLines(creds_file)
cred_list <- setNames(
  sub(".*=", "", creds),
  sub("=.*", "", creds)
)

In [4]:
con <- dbConnect(
  drv = RMariaDB::MariaDB(),
  username = cred_list[["username"]],
  password = cred_list[["password"]],
  host = "ehr3.deim.urv.cat",
  dbname = "mimiciiiv14",
  port = 3306
)

Remember that **you must first be connected to the VPN** in order to do so. Also, remember that the VPN connection is rebooted every 24h, that means that you may need to run this cell again.

The following cell can be used to check if the connection works properly. The output should be the list of tables in the database.



In [5]:
dbListTables(con)

[1] "DATETIMEEVENTS"     "MICROBIOLOGYEVENTS" "D_CPT"             
 [4] "ICUSTAYS"           "PROCEDURES_ICD"     "CHARTEVENTS"       
 [7] "SERVICES"           "CALLOUT"            "NOTEEVENTS"        
[10] "D_ITEMS"            "ADMISSIONS"         "PRESCRIPTIONS"     
[13] "INPUTEVENTS_CV"     "INPUTEVENTS_MV"     "PATIENTS"          
[16] "DRGCODES"           "D_LABITEMS"         "OUTPUTEVENTS"      
[19] "DIAGNOSES_ICD"      "PROCEDUREEVENTS_MV" "LABEVENTS"         
[22] "CAREGIVERS"         "D_ICD_DIAGNOSES"    "CPTEVENTS"         
[25] "TRANSFERS"          "D_ICD_PROCEDURES"

# SQL - relation of the exposure to a microorganism to the diagnosis of a disease

Microbiological studies are very common in the clinical practice to assess the interference of a given microorganism on the health of the patients. These studies are performed using samples cultures that after a period are analyzed to check the composition of the microorganism population.

In this exercise, we will practice some of the concepts learned on epidemiology. The objective is to **study the relation of the exposure to a microorganism to the diagnosis of a disease**. We provide a table with the following information:

-   `LONG_TITLE`: Name of the primary (`SEQ_NUM = 1`) diagnosis in long format
-   `ORG_NAME`: Name of the organism
-   `N_DIAG`: Number of admissions with the same primary diagnosis
-   `N_TESTED`: Number of admissions with the specified primary diagnosis with a microbiological study (either positive or negative)
-   `PERCENT_TESTED`: Percentage of admissions diagnosed with the same primary diagnosis tested for microorganism
-   `N_POS`: Number of admissions with at least one positive test for the presence of the specified microorganism
-   `N_NEG`: Number of admissions without any positive microbiological test for that given disease
-   `PERCENT_POS`: Percentage of tested admissions with at least a positive test for the given disease and microorganism
-   `ODDS_RATIO`: The odds ratio of the exposure on the disease following the same formulation used previously in class

The results have to be **limited** to the ones that have at least 200 positive tests on different admissions considering all the diagnoses, and the ones that have at least 50 positive tests on different admissions for the given disease. The results need to be sorted in **descending order by `N_DIAG`** and ascending order according to the odds ratio.

There are several assumptions that we need to make:

-   For simplicity, we only take into account the **primary diagnosis** (`SEQ_NUM = 1`) ignoring any possible comorbidity
-   we assume that the test applied for detecting a microorganism is **the same for all the existing types**; meaning that if we find a negative test this will indicate the no presence of any microorganism
-   we  assume that there is just one **unique microbiological test per admission**, considering all the different studies as part of the same
-   As a simplification, we may consider **all the admissions independent of the patient**
-   Remember that **the order of the JOINS matter** and their match should be unique if you wanna avoid multiple matches for the same pair of keys



In [9]:
dbExecute(con, "SET SQL_BIG_SELECTS=1")
sql <- "
/*
 * To achieve this exercise by this query helps us understand if
 * certain bacteria/organisms are related to specific diseases.
 * We re basically asking, when patients have disease X,
 * how often do they test positive for organism Y.
 *
 */

WITH primary_diagnoses AS (
	-- GET the admissionID and the LONG_TITLE from DIAGNOSES
    SELECT 
        d.HADM_ID
        ,di.LONG_TITLE
    FROM DIAGNOSES_ICD d
    INNER JOIN D_ICD_DIAGNOSES di ON d.ICD9_CODE = di.ICD9_CODE
    WHERE d.SEQ_NUM = 1
),
organism_qualified AS (
	-- Get ORG_NAME from the table MICROBIOLOGYEVENTS excluding the null values
	-- Then filter the group at least 200 positive test.
    SELECT 
        ORG_NAME
    FROM MICROBIOLOGYEVENTS
    WHERE ORG_NAME IS NOT NULL
    GROUP BY ORG_NAME
    HAVING COUNT(DISTINCT HADM_ID) >= 200
),
diag_with_positive_test AS (                            
    -- Get the information of Diagnosis-Organism 
    -- combinations with positive tests.
    -- first join the admission number to filter for any test
    -- second join to filter for qualified organism
    SELECT 
        pd.LONG_TITLE
        ,m.ORG_NAME
        ,pd.HADM_ID
    FROM primary_diagnoses pd
    INNER JOIN MICROBIOLOGYEVENTS m ON pd.HADM_ID = m.HADM_ID
    INNER JOIN organism_qualified oq ON m.ORG_NAME = oq.ORG_NAME
),
diag_with_any_test AS (
    -- Get the admissions with primary diagnosis that had any micro test
    SELECT DISTINCT
        pd.LONG_TITLE
        ,pd.HADM_ID
    FROM primary_diagnoses pd
    INNER JOIN MICROBIOLOGYEVENTS m ON pd.HADM_ID = m.HADM_ID
),
base_counts AS (
	-- Get the other results at least 50 positive tests on different
	-- admissions for the given disease.
    -- Qualified disease
    SELECT 
        dwpt.LONG_TITLE
        ,dwpt.ORG_NAME
        ,COUNT(DISTINCT dwpt.HADM_ID) AS N_POS
    FROM diag_with_positive_test dwpt
    GROUP BY dwpt.LONG_TITLE, dwpt.ORG_NAME
    HAVING COUNT(DISTINCT dwpt.HADM_ID) >= 50
),
total_tested AS (
    -- count the total number of admissions which where tested
    SELECT COUNT(DISTINCT HADM_ID) AS TOTAL_TESTED
    FROM diag_with_any_test
),
total_pos_org AS (
    -- count admissions for each specific organism.
    -- in order to calculate OR
    SELECT
        ORG_NAME,
        COUNT(DISTINCT HADM_ID) AS TOTAL_POS
    FROM diag_with_positive_test
    GROUP BY ORG_NAME
),
final_stats AS (
	-- join the first information as results of above queries.
    SELECT 
        bc.LONG_TITLE
        ,bc.ORG_NAME
        ,bc.N_POS
        ,COUNT(DISTINCT pd.HADM_ID) AS N_DIAG
        ,COUNT(DISTINCT dwat.HADM_ID) AS N_TESTED
        ,tt.TOTAL_TESTED
        ,tpo.TOTAL_POS
    FROM base_counts bc
    INNER JOIN primary_diagnoses pd ON bc.LONG_TITLE = pd.LONG_TITLE
    LEFT JOIN diag_with_any_test dwat 
        ON bc.LONG_TITLE = dwat.LONG_TITLE 
        AND pd.HADM_ID = dwat.HADM_ID
    LEFT JOIN total_pos_org tpo
        ON bc.ORG_NAME = tpo.ORG_NAME
    CROSS JOIN total_tested tt
    GROUP BY bc.LONG_TITLE, bc.ORG_NAME, bc.N_POS
)
-- Final table as output.
SELECT 
    LONG_TITLE
    ,ORG_NAME
    ,N_DIAG
    ,N_TESTED
    ,ROUND(100.0 * N_TESTED / N_DIAG, 2) AS PERCENT_TESTED
    ,N_POS
    ,(N_TESTED - N_POS) AS N_NEG
    ,ROUND(100.0 * N_POS / NULLIF(N_TESTED, 0), 2) AS PERCENT_POS
    ,ROUND(
    (N_POS *((TOTAL_TESTED - TOTAL_POS) - (N_TESTED - N_POS)))
    /
    NULLIF((TOTAL_POS - N_POS) * (N_TESTED - N_POS),0),4)
    AS ODDS_RATIO
FROM final_stats
WHERE N_TESTED > 0
ORDER BY N_DIAG DESC, ODDS_RATIO ASC;
"

[1] 0

In [ ]:

dbGetQuery(con, sql) %>% head(70)

,LONG_TITLE,ORG_NAME,N_DIAG,N_TESTED,PERCENT_TESTED,N_POS,N_NEG,PERCENT_POS,ODDS_RATIO
,<chr>,<chr>,<int64>,<int64>,<dbl>,<int64>,<int64>,<dbl>,<dbl>
1,Coronary atherosclerosis of native coronary artery,YEAST,3498,2103,60.12,85,2018,4.04,0.2901
2,Coronary atherosclerosis of native coronary artery,ENTEROCOCCUS SP.,3498,2103,60.12,53,2050,2.52,0.4038
3,Coronary atherosclerosis of native coronary artery,STAPH AUREUS COAG +,3498,2103,60.12,99,2004,4.71,0.4063
4,Coronary atherosclerosis of native coronary artery,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",3498,2103,60.12,92,2011,4.37,0.4148
5,Coronary atherosclerosis of native coronary artery,ESCHERICHIA COLI,3498,2103,60.12,67,2036,3.19,0.4768
6,"Single liveborn, born in hospital, delivered by cesarean section","STAPHYLOCOCCUS, COAGULASE NEGATIVE",2757,2243,81.36,73,2170,3.25,0.3026
7,Unspecified septicemia,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",2069,2050,99.08,225,1825,10.98,1.1564
8,Unspecified septicemia,CORYNEBACTERIUM SPECIES (DIPHTHEROIDS),2069,2050,99.08,61,1989,2.98,1.4842
9,Unspecified septicemia,ENTEROCOCCUS SP.,2069,2050,99.08,173,1877,8.44,1.5107




This is a retrospective observational analytic study, conceptually equivalent to a case–control study,
because exposure and outcome are assessed from existing clinical records and their association is quantified using odds ratios
without any experimental intervention.

### Conclusions

There is a clear pattern of testing. For infectious diagnoses (e.g. unspecified septicemia) the PERCENT_TESTED is 99–100%.
For non-infectious diagnoses (e.g. coronary atherosclerosis) the PERCENT_TESTED is 60–80%.
This reflects appropriate clinical practice. When infection is suspected, microbiological testing is almost universal.
For cardiovascular or non-infectious conditions, testing is more selective.

Non-infectious diagnoses have a low prevalence of positive tests (e.g. Coronary atherosclerosis with PERCENT_POS mostly 2–5%).
These low rates suggest incidental findings.
Infectious diagnoses have a high prevalence of positive tests (e.g. septicemia due to E. coli with PERCENT_POS 76% for E. coli).
High positivity rates align strongly with biologically plausible etiologies, validating the data and approach.

Diagnosis which come with an OR below 1 have a weak or no association with the organism (e.g. Coronary atherosclerosis with most organisms: OR ≈ 0.3–0.5).
There is no causal link.
Some organisms show OR close to 1, suggesting no strong relationship.
Very high odds ratios indicate a strong association between exposure and disease, fully consistent with known infectious causes 
(e.g. septicemia due to E. coli + E. coli: OR ≈ 51.7).

Conclusion:
The analysis demonstrates clear and biologically plausible associations between exposure to specific microorganisms and primary diagnoses.
Infectious diseases show high positivity rates for their expected pathogens, accompanied by markedly elevated odds ratios.
In contrast, non-infectious diagnoses exhibit lower positivity rates and odds ratios below one, indicating no meaningful association.
Overall, the results validate microbiological testing as a valuable epidemiological signal.

## Regular expressions

In this section, we will work with **regular expressions**. We first connect to MIMIC-III and extract the TEXT field from the NOTEEVENTS table corresponding to the only register with `SUBJECT_ID` equal to `13702`, `CATEGORY` equal to &rsquo;Discharge summary&rsquo;, and `CHARTDATE` equal to &rsquo;2118-06-14&rsquo;. Store it in a string variable called `text`.



In [15]:
sql_diagnosis_all <- "
SELECT TEXT
FROM NOTEEVENTS
WHERE SUBJECT_ID = 13702
  AND CATEGORY = 'Discharge summary'
  AND CHARTDATE = '2118-06-14'
"

diagnosis_all <- dbGetQuery(con, sql_diagnosis_all)
text <- diagnosis_all$TEXT[1]

In [ ]:

writeLines(str_c('<', text ,'>'))

<Admission Date:  [**2118-6-2**]       Discharge Date:  [**2118-6-14**]

Date of Birth:                    Sex:  F

Service:  MICU and then to [**Doctor Last Name **] Medicine

HISTORY OF PRESENT ILLNESS:  This is an 81-year-old female
with a history of emphysema (not on home O2), who presents
with three days of shortness of breath thought by her primary
care doctor to be a COPD flare.  Two days prior to admission,
she was started on a prednisone taper and one day prior to
admission she required oxygen at home in order to maintain
oxygen saturation greater than 90%.  She has also been on
levofloxacin and nebulizers, and was not getting better, and
presented to the [**Hospital1 18**] Emergency Room.

In the [**Hospital3 **] Emergency Room, her oxygen saturation was
100% on CPAP.  She was not able to be weaned off of this
despite nebulizer treatment and Solu-Medrol 125 mg IV x2.

Review of systems is negative for the following:  Fevers,
chills, nausea, vomiting, night sweats, change in w

### Medications 

In `text`, there is a paragraph with the **list of medications** on admission. Using `stringr` functions, **extract it to a list of strings called `medications_admission`**. Remove the enumeration in front of each medication. Each string in the list **is composed by the name of the medication and its dosage** just as it appears in the report.



In [ ]:

#find the paragraph of medications
med_block <- str_extract(
  text,
  regex(
    "Medications on Admission:[\\s\\S]*?(?=\\n\\n[A-Z ])",
    ignore_case = TRUE
  )
)
med_block

#extract list of strings
medications_admission <- str_extract_all(
  med_block,
    "\\n\\s*\\d+\\.\\s*[^\\n]+"
)[[1]]

#remove enumeration
medications_admission <- str_remove(
  medications_admission,
  "^\\n\\s*\\d+\\.\\s*"
)


[1] "MEDICATIONS ON ADMISSION:\n1. Hydrochlorothiazide 25 q.d.\n2. Prednisone 60 mg, 50 mg, 40 mg, 20 mg.\n3. Levofloxacin 500 mg q.d.\n4. Imdur 60 mg q.d.\n5. Synthroid 75 mcg q.d.\n6. Pulmicort nebulizer b.i.d.\n7. Albuterol nebulizer q.4. prn.\n8. Lexapro 10 mg q.d.\n9. Protonix 40 mg q.d.\n10. Aspirin 81 mg q.d."

In [ ]:

medications_admission

[1] "Hydrochlorothiazide 25 q.d."           
 [2] "Prednisone 60 mg, 50 mg, 40 mg, 20 mg."
 [3] "Levofloxacin 500 mg q.d."              
 [4] "Imdur 60 mg q.d."                      
 [5] "Synthroid 75 mcg q.d."                 
 [6] "Pulmicort nebulizer b.i.d."            
 [7] "Albuterol nebulizer q.4. prn."         
 [8] "Lexapro 10 mg q.d."                    
 [9] "Protonix 40 mg q.d."                   
[10] "Aspirin 81 mg q.d."

### Medications data frame

Now that we have extracted the medication information, we **put this information in a structured object**. Specifically, we generate a data.frame `medications` with the following columns, using regular expressions to segment the previously generated strings:

-   `medication`: name of the product
-   `dosage`: numerical value with the prescribed dosage
-   `units`: units on which the dossage is specified

Do it as precise as possible, given the variety of formats in the original text.


In [ ]:


# 1) REGEX PATTERNS
# Match a dosage WITH units, e.g. "81 mg", "75 mcg"
dose_unit_pat <- regex(
  "\\b(\\d+(?:\\.\\d+)?)\\s*(mg|mcg|g|kg|ml|mL|iu|IU|units?|%)\\b",
  ignore_case = TRUE
)

# Remove ALL occurrences of dosage+unit from the medication string
dose_unit_remove_pat <- regex(
  "\\b\\d+(?:\\.\\d+)?\\s*(mg|mcg|g|kg|ml|mL|iu|IU|units?|%)\\b\\s*[\\.,]?",
  ignore_case = TRUE
)

# Match frequency expressions (many variants)
# Examples:
#   q.d, qd, q.d.
#   b.i.d, bid
#   q.4., q4h
#   prn, p.r.n.
freq_pat <- regex(
  paste(
    "\\b(?:",
    "q\\s*\\.?\\s*d\\.?|qd\\b",                          # once daily
    "|b\\s*\\.?\\s*i\\s*\\.?\\s*d\\.?|bid\\b",           # twice daily
    "|t\\s*\\.?\\s*i\\s*\\.?\\s*d\\.?|tid\\b",           # three times daily
    "|q\\s*\\.?\\s*i\\s*\\.?\\s*d\\.?|qid\\b",           # four times daily
    "|q\\s*\\.?\\s*\\d+\\s*\\.?\\s*(?:h|hr|hrs)?\\.?\\b",# q.4., q4h, q 4 h
    "|p\\s*\\.?\\s*r\\s*\\.?\\s*n\\.?|prn\\b",           # PRN
    ")(?:\\.)?",
    sep = ""
  ),
  ignore_case = TRUE
)

# Match a dosage WITHOUT units (followed by a frequency)
# (e.g. "Hydrochlorothiazide 25 q.d.")
dose_only_pat <- regex(
  paste(
    "\\b(\\d+(?:\\.\\d+)?)\\b(?=\\s*(?:",
    "q\\s*\\.?\\s*d\\.?|qd\\b|",
    "b\\s*\\.?\\s*i\\s*\\.?\\s*d\\.?|bid\\b|",
    "t\\s*\\.?\\s*i\\s*\\.?\\s*d\\.?|tid\\b|",
    "q\\s*\\.?\\s*i\\s*\\.?\\s*d\\.?|qid\\b|",
    "q\\s*\\.?\\s*\\d+\\s*\\.?\\s*(?:h|hr|hrs)?\\.?\\b|",
    "p\\s*\\.?\\s*r\\s*\\.?\\s*n\\.?|prn\\b",
    ")\\b|\\s*$)",
    sep = ""
  ),
  ignore_case = TRUE
)


# 2) EXTRACT DOSAGE + UNITS
# First try to extract dosage WITH units
m1 <- str_match(medications_admission, dose_unit_pat)

dosage <- suppressWarnings(as.numeric(m1[, 2]))  # numeric dosage
units  <- m1[, 3]                                # units (mg, mcg, ...)

# 3) FALLBACK: DOSAGE ONLY
# If no units were found, try extracting a bare number
need_only <- is.na(dosage)

m2 <- str_match(medications_admission, dose_only_pat)

dosage[need_only] <- suppressWarnings(as.numeric(m2[need_only, 2]))
units[need_only]  <- NA_character_


# 4) CLEAN MEDICATION NAME
medication <- medications_admission |>
  # remove all "XX mg" chunks
  str_remove_all(dose_unit_remove_pat) |>
  # remove frequency expressions (q.d., b.i.d., q.4., prn, ...)
  str_remove_all(freq_pat) |>
  # clean commas and extra whitespace
  str_replace_all("\\s*,\\s*", " ") |>
  str_replace_all("\\s+", " ") |>
  # remove trailing punctuation
  str_replace_all("[\\.,;:]\\s*$", "") |>
  str_trim()

# If dosage was extracted without units,
# remove the bare number from the medication name
medication[need_only] <- medication[need_only] |>
  str_remove_all(regex("\\b\\d+(?:\\.\\d+)?\\b")) |>
  str_replace_all("\\s+", " ") |>
  str_trim()

# 5) FINAL DATA FRAME
medications <- data.frame(
  medication = medication,
  dosage     = dosage,
  units      = units,
  stringsAsFactors = FALSE
)


In [ ]:

print(medications)

            medication dosage units
1  Hydrochlorothiazide     25  <NA>
2           Prednisone     60    mg
3         Levofloxacin    500    mg
4                Imdur     60    mg
5            Synthroid     75   mcg
6  Pulmicort nebulizer     NA  <NA>
7  Albuterol nebulizer     NA  <NA>
8              Lexapro     10    mg
9             Protonix     40    mg
10             Aspirin     81    mg


**Limitations of the method**: 





This approach works well for common, simple formats but fails on more complex or uncommon ones.
It works well for:
* single doses with units (e.g. Aspirin 81 mg q.d.)
* Doses without units if a clear frequency is present (e.g. Hydrochlorothiazide 25 q.d. → dosage = 25, units = NA)
* Entries without any dose (e.g. Pulmicort nebulizer) → dosage and units set to NA

Failures:
* Multiple doses in one line (e.g. prednisone tapers): only the first dose is kept, the rest are discarded.
* Dose ranges or complex expressions (e.g. 5–10 mg, 2.5 mg/3 mL): only partial information is extracted.
* Unrecognized units or frequencies: if not covered by the regex, they may be missed or remain in the medication field.
* Medication names containing numbers (e.g. Vitamin B12): can be misinterpreted in rare edge cases.

Handling missing or multiple values:
* Missing dosage or units are stored as NA.
* Multiple dosages are reduced to a single (first) dose, causing information loss.

Overall, the method is precise for standard formats, but loses information in complex or non-standard medication descriptions.

Medications are stored on an structured table in MIMIC-III. We compare the results obtained in this exercise with the ones stored for this particular admission in MIMIC-III:


In [27]:
sql <- "
WITH admission AS (
  SELECT n.HADM_ID
  FROM NOTEEVENTS n
  WHERE n.SUBJECT_ID = 13702
    AND n.CATEGORY = 'Discharge summary'
    AND n.CHARTDATE = '2118-06-14'
)
SELECT
  p.HADM_ID,
  p.STARTDATE,
  p.ENDDATE,
  p.DRUG,
  p.DRUG_NAME_POE,
  p.DRUG_NAME_GENERIC,
  p.DOSE_VAL_RX,
  p.DOSE_UNIT_RX,
  p.FORM_VAL_DISP,
  p.FORM_UNIT_DISP,
  p.ROUTE,
  p.PROD_STRENGTH
FROM PRESCRIPTIONS p
JOIN admission a
  ON p.HADM_ID = a.HADM_ID
WHERE p.SUBJECT_ID = 13702
ORDER BY p.STARTDATE, p.DRUG;
"

dbGetQuery(con, sql)


HADM_ID,STARTDATE,ENDDATE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,PROD_STRENGTH
<int>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
107527,2118-06-02,2118-06-03,Albuterol Neb Soln,Albuterol Neb Soln,Albuterol Neb Soln,1,NEB,1,VIAL,IH,0.083%;3mL Vial
107527,2118-06-02,2118-06-14,Aspirin,Aspirin,Aspirin,81,mg,1,TAB,PO,81mg Tab
107527,2118-06-02,2118-06-14,Escitalopram Oxalate,Escitalopram Oxalate,Escitalopram Oxalate,10,mg,1,TAB,PO,10mg Tablet
107527,2118-06-02,2118-06-05,Heparin,Heparin,Heparin Sodium,5000,UNIT,1,ml,SC,5000U/ML VIAL
107527,2118-06-02,2118-06-04,Insulin,Insulin,Insulin,0,UNIT,0,VIAL,SC,Dummy Pkg for POE pump and PHA SS ordering
107527,2118-06-02,2118-06-03,Ipratropium Bromide Neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,1,NEB,1,VIAL,IH,2.5ml Vial
107527,2118-06-02,2118-06-08,Isosorbide Mononitrate (Extended Release),Isosorbide Mononitrate (Extended Release),Isosorbide Mononitrate (Extended Release),60,mg,1,TAB,PO,60mg ER tab
107527,2118-06-02,2118-06-08,Levofloxacin,Levofloxacin,Levofloxacin,500,mg,1,TAB,PO,500mg Tab
107527,2118-06-02,2118-06-14,Levothyroxine Sodium,Levothyroxine Sodium,Levothyroxine Sodium,75,mcg,1,TAB,PO,75mcg Tab



The discharge summary captures a high-level overview of medications on admission,
whereas the structured PRESCRIPTIONS table provides a detailed, time-resolved record of all medication orders
during the hospitalization. For several drugs, there is strong agreement between both sources, such as Aspirin 81 mg and more.
However, there are some differences in the drug names. For instance, Imdur in the text corresponds to Isosorbide Mononitrate
(Extended Release) in the structured table, showing brand–generic naming differences. Additionally, several inpatient or supportive medications,
such as Heparin, and Potassium Chloride, are present in PRESCRIPTIONS but absent from the discharge summary.
Overall, these examples illustrate that free-text extraction captures the clinical narrative and intent, while structured tables offer greater
completeness and precision, making them complementary rather than interchangeable sources of medication information.
